In [1]:
import tensorflow as tf
import utils
import pandas as pd
import numpy as np
__author__ = "Olivares Castillo José Luis"

tf.__version__

'1.5.0-rc0'

In [2]:
es,na = utils.load_node2vec()

In [3]:
na_dummy = na.drop(na.columns[0],axis=1)
na_vectores1 = np.array(na_dummy)

In [4]:
eval_set = pd.read_csv("../lexiconevaluacion/evaluationset",delimiter=" ",names=["esp","nah"])

In [5]:
eval_set.head()

,esp,nah
0,traer,itqui
1,traer,huica
2,traer,huiqui
3,traer,cui
4,adornar,apana


In [6]:
eval_es = list(set(eval_set["esp"]))

In [7]:
#eval_es

In [8]:
eval_es_index = [int(es[es[0] == palabra].index[0])
                  for palabra in eval_es]

In [9]:
es[es[0] == "pluma"]

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
2902,pluma,-0.386967000000000005,0.005026,-0.191887000000000002,-0.081058000000000005,-0.211675000000000002,-0.217542999999999986,-0.019026000000000001,0.454046000000000005,0.006309,...,-0.013390000000000001,0.023695999999999998,0.394602000000000008,-0.003221,-0.708667000000000047,0.021819000000000002,0.181774999999999992,0.436242000000000019,-0.162193000000000004,-0.076617000000000005


In [10]:
eval_es_index[0]

803

In [11]:
eval_es_vectores = utils.get_vectors(es,eval_es_index)

In [12]:
eval_es_vectores.shape

(150, 128)

In [13]:
test_vectors = np.array([np.array(es.iloc[indice][1::]).astype(np.float64) for indice in eval_es_index])

In [14]:
sess = tf.Session()

In [15]:
saver = tf.train.import_meta_graph('./models/model1111_gpu/model2250.ckpt.meta')
saver.restore(sess,tf.train.latest_checkpoint('./models/model1111_gpu/'))

INFO:tensorflow:Restoring parameters from ./models/model1111_gpu/model2250.ckpt


In [16]:
graph = tf.get_default_graph()

In [17]:
X = graph.get_tensor_by_name("input/input_es:0")
#y = graph.get_tensor_by_name("input/target_na:0")

In [18]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

['input/input_es',
 'input/target_na',
 'Const',
 'W1/Initializer/random_uniform/shape',
 'W1/Initializer/random_uniform/min',
 'W1/Initializer/random_uniform/max',
 'W1/Initializer/random_uniform/RandomUniform',
 'W1/Initializer/random_uniform/sub',
 'W1/Initializer/random_uniform/mul',
 'W1/Initializer/random_uniform',
 'W1',
 'W1/IsInitialized/VarIsInitializedOp',
 'W1/Assign',
 'W1/Read/ReadVariableOp',
 'Read/ReadVariableOp',
 'Identity',
 'W1/Regularizer/l2_regularizer/scale',
 'W1/Regularizer/l2_regularizer/L2Loss/ReadVariableOp',
 'W1/Regularizer/l2_regularizer/L2Loss',
 'W1/Regularizer/l2_regularizer',
 'Const_1',
 'b1',
 'b1/Assign',
 'b1/read',
 'xw_plus_b/ReadVariableOp',
 'xw_plus_b/MatMul',
 'xw_plus_b',
 'fc1',
 'Const_2',
 'b2',
 'b2/Assign',
 'b2/read',
 'transpose/Rank/ReadVariableOp',
 'transpose/Rank',
 'transpose/sub/y',
 'transpose/sub',
 'transpose/Range/start',
 'transpose/Range/delta',
 'transpose/Range',
 'transpose/sub_1',
 'transpose/ReadVariableOp',
 'trans

In [19]:
#output_NN = graph.get_tensor_by_name("output/xw_plus_b:0")#model1937
output_NN = graph.get_tensor_by_name("xw_plus_b_1:0")
#output_NN = graph.get_tensor_by_name("dense_2/BiasAdd:0")
#output_NN = graph.get_tensor_by_name("output_1:0")

In [20]:
feed_dict = {X:test_vectors}


In [21]:
pred = sess.run(output_NN,feed_dict)
print (type(pred[0]),pred.shape)

<class 'numpy.ndarray'> (150, 128)


In [22]:
top_10=[utils.get_top10_closest(pred[_],na_vectores1) for _ in range(pred.shape[0])]

In [23]:
closest = [utils.get_closest_words_to(top_10[_],na) for _ in range(pred.shape[0])]

In [24]:
es[es[0]=="adquisición"].index

Int64Index([2222], dtype='int64')

In [25]:
na[na[0]=="v"].index

Int64Index([1376], dtype='int64')

In [26]:
eval_es[6]

'cargo'

In [27]:
closest[6]

['ahqui',
 'tlaliliz',
 'chihui',
 'onino',
 'xip',
 'notelpo',
 'ticmo',
 'milacatzo',
 'im',
 'mix']

In [28]:
resultados = {palabra_es:top_10_nah for (palabra_es,top_10_nah) in zip(eval_es,closest)}

In [29]:
esp = list(eval_set["esp"].values)

In [30]:
nah = list(eval_set["nah"].values)

In [31]:
pares_eval = list(zip(esp,nah))

In [32]:
from collections import defaultdict

In [33]:
gold = defaultdict(list)
for palabra_es,palabra_na in pares_eval:
    gold[palabra_es].append(palabra_na)


In [34]:
gold = dict(gold)

In [35]:
gold["amado"]

['tlazo', 'tlazohtla']

In [36]:
resultados["amado"]

['tlazohtla',
 'ticcua',
 'ama',
 'zquia',
 'tlazo',
 'diego',
 'comal',
 'niz',
 'secretario',
 'federal']

In [37]:
p1 = 0
p5 = 0
p10 = 0
list_esp_eval = (list(resultados.keys()))
hits=list()

not_found = list()


for palabra_gold in list_esp_eval:
    for i in gold[palabra_gold]:
        if i in resultados[palabra_gold]:
            hits.append(resultados[palabra_gold].index(i))
    if hits.__len__() > 0:
        if min(hits) == 0:
            p1 += 1
            p5 += 1
            p10 += 1
        if min(hits) >= 1 and min(hits) <= 5:
            p5 += 1
            p10 += 1
        if min(hits) > 5 and min(hits) <= 10:
            p10 += 1
        print(palabra_gold,min(hits),hits,p1,p5,p10)
    else:
        not_found.append(palabra_gold)
        print(palabra_gold+": NOT FOUND")
   
    hits.clear()

length=list_esp_eval.__len__()
print("\nP@1:",p1/length,"\tP@5:",p5/length,"\tP@10:",p10/length)
     


llevar 0 [0, 1, 4] 1 1 1
alfredo 0 [0] 2 2 2
acabar 0 [0] 3 3 3
patio 0 [0, 1, 2] 4 4 4
oír 2 [2] 4 5 5
ropa 0 [0] 5 6 6
cargo: NOT FOUND
admirar 0 [0, 2] 6 7 7
pipián 0 [0, 7] 7 8 8
paso 0 [0] 8 9 9
teatro 0 [0, 2] 9 10 10
levantar 0 [0] 10 11 11
comenzar 0 [0, 1] 11 12 12
adquisición 8 [8] 11 12 13
vasija 0 [1, 3, 0] 12 13 14
sólo 0 [0] 13 14 15
itacates 0 [0] 14 15 16
decir 0 [1, 3, 0] 15 16 17
nido: NOT FOUND
volver 0 [0, 1] 16 17 18
agregar 0 [0, 4] 17 18 19
quiltoniles 9 [9] 17 18 20
tigre 0 [0, 1] 18 19 21
indígena 1 [1] 18 20 22
derecho 1 [1, 2] 18 21 23
descanso 0 [0] 19 22 24
enorme: NOT FOUND
salvador 0 [0, 1, 2] 20 23 25
puramente: NOT FOUND
querer 0 [1, 0, 9, 2] 21 24 26
tanto 1 [1] 21 25 27
amado 0 [4, 0] 22 26 28
echar 1 [1] 22 27 29
presidente 0 [0] 23 28 30
traer 1 [1, 2] 23 29 31
tepaneca 1 [1] 23 30 32
olvidar 4 [4] 23 31 33
grande 0 [1, 0] 24 32 34
centenario 0 [0, 4] 25 33 35
reunir 1 [1] 25 34 36
limpiar 0 [1, 0, 5] 26 35 37
indio 1 [1] 26 36 38
destrozar: NOT FOU

In [38]:
not_found.__len__()

16

In [39]:
len(list(resultados.keys()))

150

In [40]:
resultados_gold = dict()
for k,v in resultados.items():
    resultados_gold[k] = {"GOLD":gold[k],"RESULTS":v}

In [41]:
na[na[0]=="español"]

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
1342,español,-0.059672000000000003,-0.354704000000000019,-0.177631000000000011,0.082270999999999983,0.294655,0.32784000000000002,-0.148658000000000012,-0.437325999999999993,-0.064614000000000019,...,0.145312999999999998,0.263815000000000022,-0.113244000000000011,0.194174999999999986,0.199001000000000011,0.957316999999999974,-0.743376999999999843,0.100334000000000007,0.050588000000000001,-0.401527000000000023


In [42]:
for res in not_found:
    print(res+":",resultados_gold[res],end="\n"*2)

cargo: {'GOLD': ['tlahtoca', 'toca'], 'RESULTS': ['ahqui', 'tlaliliz', 'chihui', 'onino', 'xip', 'notelpo', 'ticmo', 'milacatzo', 'im', 'mix']}

nido: {'GOLD': ['tapazolli', 'tapazol', 'pa', 'zol'], 'RESULTS': ['nepapan', 'motemilih', 'mota', 'huipan', 'tenanco', 'diego', 'dral', 'tlazo', 'secretario', 'rosario']}

enorme: {'GOLD': ['huey'], 'RESULTS': ['mil', 'huehca', 'cpac', 'tepahpaquiltia', 'dral', 'occe', 'oc', 'zcuco', 'diego', 'jard']}

puramente: {'GOLD': ['zan', 'za'], 'RESULTS': ['ommihzo', 'imayauhcan', 'imayauhcampa', 'ceuh', 'nepantla', 'dral', 'secretario', 'diego', 'tepahpaquiltia', 'pochcopa']}

destrozar: {'GOLD': ['teciz', 'tlanteciz'], 'RESULTS': ['x', 'tepahpaquiltia', 'yb', 'dral', 'fa', 'diego', 'rosario', 'xip', 'secretario', 'zcuco']}

efecto: {'GOLD': ['nel'], 'RESULTS': ['tlaocox', 'nehnemi', 'ihtotia', 'oquich', 'mixco', 'tz', 'oceloquichtle', 'iuhquin', 'icnihuan', 'xuchitl']}

final: {'GOLD': ['tlatz'], 'RESULTS': ['mix', 'xip', 'im', 'yb', 'dral', 'mochi'

In [43]:
resultados_gold["sabor"]

{'GOLD': ['iahhuui', 'mati'],
 'RESULTS': ['mati',
  'lan',
  'tro',
  'chil',
  'iahhuui',
  'xinach',
  'ahnozo',
  'choh',
  'ixmati',
  'yb']}

In [44]:
for k,v in resultados_gold.items():
    print("Palabra:",k.upper(),"\nGOLD:",v["GOLD"],"\nRESULTADOS:",
          v["RESULTS"],end="\n"*2)

Palabra: LLEVAR 
GOLD: ['huica', 'cuica', 'tqui'] 
RESULTADOS: ['huica', 'cuica', 'mama', 'polactia', 'tqui', 'tepahpaquiltia', 'xip', 'cui', 'zcuco', 'quiliz']

Palabra: ALFREDO 
GOLD: ['alfredo'] 
RESULTADOS: ['alfredo', 'matiloa', 'chachal', 'felipe', 'ieltapach', 'xacualoa', 'club', 'rafael', 'organil', 'salveros']

Palabra: ACABAR 
GOLD: ['tlami', 'tami'] 
RESULTADOS: ['tlami', 'mic', 'cauh', 'ahui', 'miqui', 'auh', 'totecuyo', 'yoltequipacho', 'xip', 'tepahpaquiltia']

Palabra: PATIO 
GOLD: ['patio', 'itual', 'ithual'] 
RESULTADOS: ['patio', 'itual', 'ithual', 'nepantla', 'ber', 'calitic', 'pixca', 'cotona', 'ticatca', 'ommihzo']

Palabra: OÍR 
GOLD: ['cac', 'caqui'] 
RESULTADOS: ['acica', 'nlcaci', 'cac', 'cuic', 'ami', 'ihuinti', 'camati', 'quimolhui', 'ihiyo', 'tlahtoani']

Palabra: ROPA 
GOLD: ['tzotzoma'] 
RESULTADOS: ['tzotzoma', 'ui', 'nuevo', 'xmatia', 'xolochcue', 'dral', 'lahuan', 'yancui', 'secretario', 'tzatzapal']

Palabra: CARGO 
GOLD: ['tlahtoca', 'toca'] 
RESULTAD

GOLD: ['ramos'] 
RESULTADOS: ['ramos', 'g', 'señor', 'vor', 'poalli', 'pixca', 'ri', 'gr', 'egorio', 'pualli']

Palabra: NABO 
GOLD: ['bo'] 
RESULTADOS: ['stancia', 'bo', 'catarina', 'ga', 'cuechahua', 'cec', 'acacic', 'chicauh', 'chipahuaque', 'chilmolli']

Palabra: SENO 
GOLD: ['xillan'] 
RESULTADOS: ['xillan', 'notozca', 'motozcatlan', 'illam', 'itoz', 'immoma', 'uitlapilco', 'mocnelili', 'cuitlaxcol', 'potztlatziuh']

Palabra: GRITAR 
GOLD: ['tzatzi', 'tzahtzi'] 
RESULTADOS: ['tzatzi', 'ihcahuaca', 'lero', 'tzahtzi', 'tlacacalata', 'cirio', 'ictique', 'huixoa', 'cozca', 'laxoc']

Palabra: ROSTRO 
GOLD: ['mixco'] 
RESULTADOS: ['mixco', 'tlachia', 'yollo', 'timamalihuac', 'notelpo', 'mix', 'notozca', 'ixeo', 'naz', 'chihui']

Palabra: ESTABLECER 
GOLD: ['tlalia', 'chan'] 
RESULTADOS: ['itoa', 'cauh', 'iuh', 'ixpetla', 'lique', 'cala', 'tepe', 'huellala', 'lnami', 'auh']

Palabra: FAMILIAR 
GOLD: ['familiar'] 
RESULTADOS: ['familiar', 'xicol', 'tlaloa', 'cocox', 'chmotlahpalhui', 'chi